#### Monday, December 23, 2024

Running 'lmstudio-community/Qwen2.5-14B-Instruct-GGUF' through LMStudio.

This all runs in one pass.

In [3]:
# Deliberately set the OPENAI_API_KEY to an invalid value to ensure that the code is not using it.
import os
os.environ['OPENAI_API_KEY'] = "Nope!"

In [1]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.output_parsers.pydantic import PydanticOutputParser
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
)
from pydantic.v1 import BaseModel, Field
from typing import List

In [2]:
class Query(BaseModel):
    id: int
    question: str
    dependencies: List[int] = Field(
        default_factory=list,
        description="""A list of sub-queries that must be completed before this task can be completed. 
        Use a sub query when anything is unknown and we might need to ask many queries to get an answer. 
        Dependencies must only be other queries."""
    )

class QueryPlan(BaseModel):
    query_graph: List[Query]

In [4]:
# model = ChatOpenAI()

from langchain_openai.chat_models import ChatOpenAI
import json
from os import getenv

# 2070 Super
api_key = "LMStudio"
lmstudio = "http://127.0.0.1:1234/v1"
model = "lmstudio-community/Qwen2.5-14B-Instruct-GGUF"

# chat = ChatOpenAI(base_url=lmstudio, model=model, api_key="LMStudio",  model_kwargs={"response_format": {"type": "json_object"}})
model = ChatOpenAI(base_url=lmstudio, api_key=api_key)

In [ ]:
# Set up a parser:
parser = PydanticOutputParser(pydantic_object=QueryPlan)

In [ ]:
template = """Generate a query plan. This will be used for task execution.

Answer the following query: {query}

Return the following query graph format:
{format_instructions}
"""

In [5]:
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])

# Create the LCEL chain with the prompt, model and parser:
chain = chat_prompt | model | parser

In [ ]:
result = chain.invoke({
"query":'''I want to get the results from my database. Then I want to find out what the average age of my top 10 customers is.  
    Once I have the average age, I want to send an email to John. Also I just generally want to send a welcome introduction email to Sarah, 
    regardless of the other tasks.''',
"format_instructions":parser.get_format_instructions()
})

print(result.query_graph)

# 26.2 s

# Expected output:
# [Query(id=1, question='Get results from database', dependencies=[]), Query(id=2, question='Find average age of top 10 customers', dependencies=[1]), Query(id=3, question='Send email to John', dependencies=[2]), Query(id=4, question='Send welcome introduction email to Sarah', dependencies=[])]

[Query(id=1, question='Get the results from the database.', dependencies=[]), Query(id=2, question='Find out what the average age of my top 10 customers is.', dependencies=[1]), Query(id=3, question='Send an email to John with the average age of my top 10 customers.', dependencies=[2]), Query(id=4, question='Send a welcome introduction email to Sarah.', dependencies=[])]
